### 《人工智能》课程设计一
#### 实验思路：（该设计是在13周前完成的，当时并未告知只能使用Mindsp框架，相关情况老师已经知晓，请见谅）

1.使用tensorflow中的keras作为训练框架

2.使用MobileNetV2作为迁移学习的基础模型

3.在初次训练基础上，进行微调

4.使用随机旋转、水平反转图像，随机重置神经元的方式，缓解过拟合问题
### 实验步骤

In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

#### 读取数据并进行预处理

使用高级 Keras 预处理效用函数tf.keras.utils.image_dataset_from_directory

为加载器定义一些参数并加载训练、验证数据集
##### （train文件夹，val文件夹，test文件夹，统一放在了data文件夹中）

In [7]:
train_directory = 'data/train'  
val_directory = 'data/val'
batch_size = 32            #批量大小
img_size = (160, 160)

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_directory,
    shuffle=True,
    image_size=img_size,
    batch_size=batch_size
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_directory,
    shuffle=True,
    image_size=img_size,
    batch_size=batch_size
)

# 提取类别名
class_names = train_dataset.class_names
print(f"Class Names: {class_names}")

Found 1400 files belonging to 10 classes.
Found 200 files belonging to 10 classes.
Class Names: ['SUV', 'bus', 'family sedan', 'fire engine', 'heavy truck', 'jeep', 'minibus', 'racing car', 'taxi', 'truck']


#### 配置数据集以提高性能

使用缓冲预提取从磁盘加载图像，以免造成 I/O 阻塞。

In [8]:
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

#### 使用数据扩充

由于提供的数据集规模不大，引入随机旋转和水平翻转，增加数据的多样性。有助于缓解过拟合的问题

In [9]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.0005),
])

#### 重新缩放像素值

使用tf.keras.applications.MobileNetV2 作为基础模型。此模型期望像素值处于 [-1, 1] 范围内，但此时，图像中的像素值处于 [0, 255] 范围内。要重新缩放这些像素值，此时使用模型随附的预处理方法。

In [10]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

#### 从预训练卷积网络创建基础模型

根据 Google 开发的 MobileNet V2 模型来创建基础模型。此模型已基于 ImageNet 数据集进行预训练，ImageNet 数据集是一个包含 140 万个图像和 1000 个类的大型数据集。ImageNet 是一个研究训练数据集，具有各种各样的类别。

将 MobileNet V2 用于展平操作之前的最后一层，用于特征提取。此层被称为“瓶颈层”。与最后一层/顶层相比，瓶颈层的特征保留了更多的通用性。

首先，实例化一个已预加载基于 ImageNet 训练的权重的 MobileNet V2 模型。通过指定 include_top=False 参数，可以加载不包括顶部分类层的网络，这对于特征提取十分理想。

其次，在编译和训练模型之前，冻结卷积基至关重要。冻结（通过设置 layer.trainable = False）可避免在训练期间更新给定层中的权重。

In [11]:
img_shape = img_size + (3,)  # 添加通道数
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_shape,
    include_top=False,  # 不加载顶层分类器
    weights='imagenet'  
)
base_model.trainable = False

#### 添加分类头

要从特征块生成预测，使用 tf.keras.layers.GlobalAveragePooling2D 层在 5x5 空间位置内取平均值，以将特征转换成每个图像一个向量（包含 1280 个元素）。

预测的输出数为10，使用softmax，将输入的未归一化的数值转换为概率分布。

In [13]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(class_names), activation='softmax')  

通过使用 Keras 函数式 API 将数据扩充、重新缩放、base_model 和特征提取程序层链接在一起来构建模型。如前面所述，由于模型包含 BatchNormalization 层，因此使用 training = False。训练模型过程中，使用data_augmentation对数据进行增强；preprocess_input对数据进行预处理，及转换成mobilenetV2的输入格式；base_model将输入数据传入预训练的基础模型（mobilenetV2），并进行特征提取；global_average_layer对基础模型的输出进行全局平均池化，通过这种方式，模型可以得到一个更加紧凑的特征表示，用作于替代全连接层；Dropout 是一种正则化技术，通过在训练时随机忽略部分神经元（通过置零），避免模型过拟合，它有效地减少了神经网络对某些神经元的依赖，使得模型更具泛化能力。

In [14]:
inputs = tf.keras.Input(shape=img_shape)
x = data_augmentation(inputs)      #数据扩充
x = preprocess_input(x)  # [0,255]--->[-1,1]
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)  # 随机重置20%神经元，防止过拟合
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

#### 编译模型

使用Adam优化器，SparseCategoricalCrossentropy作为分类问题的交叉熵损失函数，指定评估指标。

In [ ]:
base_learning_rate=0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [18]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 160, 160, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 160, 160, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                             

MobileNetV2 是一种预训练的深度学习模型，它的最后一个可用的卷积层输出大小是 (None, 5, 5, 1280)，即每张输入图片经过卷积运算后，变成了一个 5x5 的特征图，并且有 1280 个通道。这是 MobileNetV2 的默认设计，无论调整输入大小如何，它都会将高层特征压缩到 1280 个通道。

在模型中，Dense 层的作用是将从 GlobalAveragePooling2D 层得到的 (None, 1280) 输入映射到 (None, 10)，即最终的分类输出。Dense 层的参数计算如下：
##### (1280+1)*10=12810
加 1 是因为每个输出神经元还有一个偏置（bias）参数。


#### 训练模型

经过实验，15为较好的循环值

In [ ]:
initial_epochs = 15
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=initial_epochs
)

#### 学习曲线

我们看一下使用 MobileNet V2 基础模型作为固定特征提取程序时训练和验证准确率/损失的学习曲线。

In [ ]:
result_directory='result'

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,3.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.savefig(os.path.join(result_directory, f"学习曲线.png"))

![学习曲线](result/学习曲线.png)

验证指标明显优于训练指标，主要原因是 tf.keras.layers.BatchNormalization 和 tf.keras.layers.Dropout 等层会影响训练期间的准确率。在计算验证损失时，它们处于关闭状态。

#### 微调

由上图可知，此时训练已经到达瓶颈，此时可以采用微调操作。

在特征提取实验中，仅在 MobileNet V2 基础模型的顶部训练了一些层。预训练网络的权重在训练过程中未更新。

进一步提高性能的一种方式是在训练（或“微调”）预训练模型顶层的权重的同时，同时训练添加的分类器。训练过程将强制权重从通用特征映射调整为专门与数据集相关联的特征。

另外，还应尝试微调少量顶层而不是整个 MobileNet 模型。在大多数卷积网络中，层越高，它的专门程度就越高。前几层学习非常简单且通用的特征，这些特征可以泛化到几乎所有类型的图像。随着您向上层移动，这些特征越来越特定于训练模型所使用的数据集。微调的目标是使这些专用特征适应新的数据集，而不是覆盖通用学习。

#### 解冻模型的顶层

解冻90层以上的层

In [22]:
base_model.trainable = True
fine_tune_at = 90  # 微调的层数
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

#### 编译模型

此阶段应使用较低的学习率。否则，模型可能会很快过拟合。

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [23]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 160, 160, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 160, 160, 3)       0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 160, 160, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 160, 160, 3)      0         
 a)                                                              
                                                                 
 mobilenetv2_1.00_160 (Funct  (None, 5, 5, 1280)       2257984   
 ional)                                                          
                                                             

#### 继续训练模型

共训练10个循环

In [ ]:
fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

history_fine = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1]
)

#### 微调曲线

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.5])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')

plt.savefig(os.path.join(result_directory, f"微调学习曲线.png"))

![微调曲线](result/微调曲线.png)

可以看到验证准确率由88%提高到92%，训练准确率也从90%提高到97%。但是训练了21个循环后，出现了验证损失大于训练损失的情况，说明可能出现过拟合的情况。因此训练设定为25个循环，防止过度拟合的加剧。


#### 保存模型

In [ ]:
model.save('car_classification.keras')


#### 评估和预测

In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import numpy as np
model = tf.keras.models.load_model('car_classification.keras')   #加载模型

test_images=[]
test_directory = 'data/test'
img_size = (160, 160)
batch_size = 32
result_directory = 'result'

class_names = ['SUV', 'bus', 'family sedan', 'fire engine', 'heavy truck', 'jeep', 'minibus', 'racing car', 'taxi', 'truck']

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_directory,
    labels=None,
    shuffle=False,
    image_size=img_size,
    batch_size=batch_size
)

predictons = model.predict(test_dataset)

images = []
for batch in test_dataset:
    for img in batch:
        images.append(img.numpy())

for i in range(0, len(predictons), 25):
    plt.figure(figsize=(20, 20))
    for j in range(25):
        plt.subplot(5, 5, j + 1)
        plt.imshow(images[i + j].astype("uint8"))
        plt.title(f"Predicted: {class_names[np.argmax(predictons[i + j])]}")
        plt.axis("off")
    plt.savefig(os.path.join(result_directory, f"result_{int(i/25+1)}.png"))
    plt.close()

#### 数据分析

##### 根据下图，可统计出200张照片中，共有12张图片识别错误，准确率为94%，与训练时计算的准确度吻合。
##### （对图像进行了图画，格式由.png自动转换成.jpeg)

![数据集一](result/result_1.jpeg)
![数据集二](result/result_2.jpeg)
![数据集三](result/result_3.jpeg)
![数据集四](result/result_4.jpeg)
![数据集五](result/result_5.jpeg)
![数据集六](result/result_6.jpeg)
![数据集七](result/result_7.jpeg)
![数据集八](result/result_8.jpeg)


